### json 불러와서 저장하기

In [1]:
import numpy as np
import pandas as pd
import json
import jpype
from bs4 import BeautifulSoup as bs

data_path = "data(test)"

json_train=[]
for i in range(0,18):
    json_file = json.loads(open("%s/%d.json" % (data_path, i)).read())
    json_train += json_file["articles"]

FileNotFoundError: [Errno 2] No such file or directory: 'data(test)/1.json'

### json에서 text 형태소 추출

In [3]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    text = bs(article["text"],"lxml").text
    
    if cnt % 10000 == 0:
        print(cnt)

    labeled_train.append({
            "istroll": article["is_troll"],
            "text_pos": ["%s_%s" % (first, second) for first, second in mecab.pos(text)]
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

###bag of words

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
def make_bag_of_words(labeled_train, max_features, col_name):
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                stop_words = None, max_features=max_features)

    train_data_features = vectorizer.fit_transform((labeled_train[col_name].apply(lambda x:" ".join(x)))).toarray()

    col = ["bow_%s_%s" % (col_name, data) for data in vectorizer.get_feature_names()]
    labeled_train = pd.concat([labeled_train, pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)],axis=1)
    
    return labeled_train

##LDA

In [4]:
import gensim
from gensim import corpora, models
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
from nltk.corpus import stopwords
from optparse import OptionParser

def make_lda(train, keep_n, num_topics, col_name):

    data = train[col_name]

    dictionary = corpora.Dictionary(data)
    dictionary.filter_extremes(keep_n=keep_n)

    corpus = [dictionary.doc2bow(text) for text in data]

    print("Make Lda..")

#    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1,distributed = True)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1)
    
    num = len(train)    
    df = []
    for i in range(0,num):
        if i%10000 == 0: print(i)
        temp = [i[1] for i in lda.get_document_topics(corpus[i],minimum_probability=0)]
        #temp = max(lda.get_document_topics(corpus[i]),key=lambda item:item[1])[0]
        df.append(temp)
    
    col = ["lda_%s_%d" % (col_name, data) for data in range(0, num_topics)]
    df = pd.DataFrame(df, columns = col)
    df.index = index
    
    train = pd.concat([train,df],axis=1)
        
    return train

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [5]:
max_features = 1000
%time labeled_train = make_lda(labeled_train, 5000, 1000, "text_pos")

TypeError: list indices must be integers or slices, not str

### predictor, model 생성

In [6]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['text_pos', label])

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

AttributeError: 'list' object has no attribute 'columns'

###cross validation

In [7]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")
cv_result = scores.mean()

print(cv_result)

NameError: name 'model' is not defined

##Result
  * text의 morphs를 bag of words로 변환 (feature : 1,000개) - 0.637526911627
  * text의 pos를 bag of words로 변환 (feature : 1,000개) - 0.661265083065
  * text의 pos를 lda로 변환 (keep_n : 5,000개, num_topics : 1,000개) - 0.643143549702
  * bs의 파라미터를 "lxml"이 아닌 "html.parser"로 주고 pos를 bag of words로 변환 (feature : 1,000개) - 0.657726939833

In [9]:
import pickle

pickle.dump(labeled_train[pre], open("text_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)